# Práctica 2: Manejo básico de ROS2

# Integrantes

* **Avilés Grimaldo Daniel**
* **Zavala López Carlos Eduardo**
* **Estrada López Pablo**

## Objetivo

Que el alumno haga uso de las funciones básicas de ROS2 (publicador y suscriptor) y de sus aplicaciones con turtlesim y rviz2

### Metas

Que el alumno:
- Integre texto en Markdown dentro de Jupyter
- Integre código de Python dentro de Jupyter
- Haga uso de la terminal de Ubuntu y contruya un script de bash
- Genere un repositorio en GitHub y comparta su archivo de práctica

### Contribución al perfil del egresado

La siguiente práctica contribuye en los siguientes puntos al perfil del egresado:

#### Aptitudes y habilidades

- Para diseñar, construir, operar y mantener los sistemas mecatrónicos y sus componentes.
- Para crear, innovar o evaluar las tecnologías relacionadas con la mecatrónica.

#### Actitudes profesionales

- Ser creativo e innovador.
- Tener confianza en su preparación académica.
- Comprometido con su actualización, superación y competencia profesional.

#### Actitudes de tipo social

- Promover el cambio en la mentalidad frente a la competitividad internacional.

## Rúbrica de evaluación

### La evaluación de la práctica contará de los siguientes puntos

| Elemento | Porcentaje |
| ------:| ----------- |
| **Cuestionario previo** | 15% | 
| **Desarrollo** | 35% |
| **Análisis de resultados**  | 35% |
| **Conclusiones** | 15% |

### Se evaluará con los siguientes criterios:

| Elemento | Malo | Regular | Bueno |
| ------:| ------ | --------| ------|
| **Cuestionario previo** | El trabajo no contiene cuestionario previo o todas las preguntas son incorrectas (0%)| Al menos la mitad de las preguntas son correctas (8%) |  Todas las preguntas son correctas (15%) |
| **Desarrollo** | El trabajo no contiene desarrollo o su planteamiento no concuerda con lo deseado (0%) | El desarrollo está mal planteado o no llega a los resultados esperados (10%) | El desarrollo tiene un planteamiento adecuado y llega a los resultados esperados (35%) |
| **Análisis de resultados**  | El trabajo no contiene análisis de resultados o la información no se está interpretando correctamente (0%) | La interpretación de los resultados es parcial o desorganizada (10%) | Realiza un correcto análisis de los resultados de forma organizada   (35%) |
| **Conclusiones** | El trabajo no contiene conclusiones o no hacen referencia al trabajo desarrollado y los objetivos planteados (0%) | La redacción de las conclusiones es desorganizada o confusa (8%) | Las conclusiones del trabajo son claras y hacen referencia al trabajo desarrollado y los objetivos planteados (15%) | 

## Introducción

### ROS2

ROS 2 (Robot Operating System 2) es un conjunto de bibliotecas y herramientas que permiten desarrollar software para robots de forma modular, distribuida y escalable.
En ROS 2, los programas se comunican entre sí mediante mensajes que viajan sobre una red (incluso entre computadoras distintas), usando un sistema de comunicación llamado DDS (Data Distribution Service).

<img src="imagenes/ROS2_logo.webp" alt = "ROS2" height="100" display= "block"/>

#### Nodos

Un nodo es la unidad básica de ejecución en ROS 2.
Cada nodo es un programa que realiza una tarea específica dentro del sistema del robot: leer un sensor, controlar un motor, procesar imágenes, etc.
En Python, los nodos se definen a partir de la clase base rclpy.node.Node.
~~~python
from rclpy.node import Node

class Nodo(Node):
    def __init__(self):
        super().__init__('nombre_nodo')
        self.get_logger().info("Nodo iniciado")
~~~

#### Tópicos

Los tópicos (topics) son canales de comunicación asíncrona y continua entre nodos.
Un nodo publica mensajes en un tópico, y otros nodos se suscriben para recibirlos.
Todos los mensajes que viajan por un tópico tienen un tipo de dato definido, por ejemplo:
- std_msgs/msg/String
- geometry_msgs/msg/Twist

Por ejemplo:

- Un nodo A (sensor) publica temperaturas en el tópico /temperatura.
- Otro nodo B (monitor) se suscribe a /temperatura y muestra los valores.

Para generar un publicador, se crea un publicador definiendo el tipo de mensaje y el tópico por el que se publica. Después se manda a llamar al publicador para mandar un mensaje.
~~~python
  topic_name = "/topic"
  self.publisher = self.create_publisher(String, topic_name, 10)

  msg = String()
  msg.data = "Message"
  self.publisher.publish(msg)
~~~
Para generar un subscriptor:
~~~python 
  topic_name = "/topic"
  self.subscription = self.create_subscription(
    String, topic_name, self.listener_callback, 10)
~~~

#### Servicios

Los servicios (services) son una forma de comunicación sincrónica (de solicitud–respuesta). Funcionan como una llamada a función entre nodos: uno envía una solicitud (request), y el otro responde con un resultado (response).

Los servicios se usan para operaciones puntuales, no continuas (por ejemplo, mover un robot a una posición, tomar una foto, guardar datos, etc.).

Por ejemplo:

- Un nodo A (cliente) pide: “calcula la suma de 2 y 3”.
- Otro nodo B (servidor) responde: “la suma es 5”.

Para crear un servidor, hay que crear el objeto y la función que se llama al recibir una solicitud

~~~python
  self.srv = self.create_service(AddTwoInts, server_name, self.add_two_ints_callback)
def add_two_ints_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
  response.sum = request.a + request.b
  self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
  return response
~~~

Para crear un cliente, hay que crear el objeto del cliente y la solicitud, enviarla, y definir la función que se llamará al recibir el objeto de la respuesta
~~~python
self.client = self.create_client(AddTwoInts, service_name)
# Esperamos al servicio
while not self.client.wait_for_service(timeout_sec=1.0):
  self.get_logger().info('Esperando al servicio {}...'.format(service_name))
request = AddTwoInts.Request()
request.a = self.a
request.b = self.b
# --- Manda solicitud
future = self.client.call_async(request)
# --- Función que se ejecuta al recibir respuesta
future.add_done_callback(self.callback_result)
~~~

#### Parámetros

Los parámetros son valores configurables que un nodo puede leer o modificar durante su ejecución.
Permiten ajustar el comportamiento del nodo sin cambiar el código.

Por ejemplo:

- Un nodo de control de motores puede tener un parámetro max_velocity de 0.1.
- Otro nodo puede cambiarlo a 0.5 sin interrumpir al primer nodo.

Por ejemplo, si en un nodo queremos que el tópico utilizado sea un parámetro
~~~python
self.declare_parameter("topic_param", "/topic")
topic_name = self.get_parameter("topic_param").value
~~~

Una forma gráfica de ver las comunicaciones en ROS2:

<img src="imagenes/Nodes-TopicandService.gif" height = "400" display = "block" alt = "Vosualización de las comunicaciones en ROS2"/>

### URDF (Unified Robot Description Format):

El URDF es un formato de archivo XML que se utiliza para describir la estructura de un robot, comunmente usado en ROS. Permite definir la composición física de un robot, incluyendo sus eslabones y juntas. Además, permite especificar características como el peso, la geometría, y la disposición espacial de los elementos del robot.

Un archivo URDF básico se construye a partir de **eslabones** y **juntas**.

#### Para definir un eslabón, se utiliza la estructura siguiente:
~~~xml
<!--Definición del eslabón-->
<link name = "nombre_del_eslabón">
  <!--
  Elementos visuales del eslabón 
  (modelos o formas geométricas)
  -->
  <visual>
      <!--
      Posición y rotación del elemento visual
      respecto al origen del eslabón
      -->
      <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
        <!--Elementos visuales asociados a la junta-->
        <geometry> 
        <!--Primitiva visual (caja en este caso)-->
        <box size = "x y z"/> 
     </geometry>
        <!--Material usado para el elemento visual-->
        <material name = "nombre_del_material"> 
          <color rgba = "r g b a"/> 
        </material> 
  </visual> 
</link> 
~~~

Los elementos primitivos son formas geométricas básicas que se utilizan para representar las partes de un robot en URDF. Algunos de los más comunes son:

- **Caja (box)**: Un paralelepípedo definido por su ancho, alto y profundidad.
>~~~xml
>  <box size = "x y z"/> 
>~~~
- **Cilindro (cylinder)**: Definido por su radio y longitud.
>~~~xml
>  <cylinder radius = "r" length="l"/> 
>~~~
- **Esfera (sphere)**: Definida por su radio.
>~~~xml
>  <sphere radius = "r"/> 
>~~~
- **Malla (mesh)**: Permite importar geometrías más complejas en formatos como STL, Collada, obj, entre otros.
>~~~xml
>  <mesh filename = "ruta/al/archivo"/> 
>~~~

Se puede agregar cómo se desplegarán estos elementos a través de un **material**: 
- **Color**: Color plano con el que se despliega el elemento, definido por componentes rgb y transparencia (alfa).
>~~~xml
> <material name = "Nombre"> 
>   <color rgba = "r g b a"/> 
> </material> 
>~~~
- **Textura**: Imagen que se despliega sobre el modelo.
>~~~xml
> <material name = "Nombre"> 
>   <texture filename = "ruta/al/archivo"/> 
> </material> 
>~~~

#### Para una junta: 
~~~xml
<!--Definición de la junta-->
<joint name = "nombre_de_la_junta" type = "tipo_de_junta"> 
  <!--
  Eslabones padre e hijo
  - Padre: Eslabón respecto al cual se mide la rotació o traslación
  - Hijo: Eslabón que se desplaza o rota
  -->
  <parent link = "eslabon_padre_(fijo)"/> 
  <child link = "eslabon_hijo_(movible)"/> 
  <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
  <!--Eje de rotación  (1: se mueve. 0: no se mueve)-->
  <axis xyz = "x y z"/> 
  <!--Límites 
  - effort: par máximo
  - velocity: velocidad absoluta máxima
  - lower-upper: posición mínima y máxima-->
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
~~~
En el formato URDF, existen varios tipos de juntas que pueden definir el movimiento relativo entre las partes de un robot:

- **Junta fija**: No permite ningún movimiento entre los elementos conectados.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "fixed"> 
>~~~
- **Junta rotacional**: Permite un movimiento de rotación alrededor de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "revolute"> 
>~~~
- **Junta prismática**: Permite un movimiento lineal a lo largo de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "prismatic"> 
>~~~
- **Junta continua**: Es una variación de la junta rotacional que permite una rotación sin límites.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "continuous"> 
>~~~

### RVIZ


RViz (ROS Visualization) es una herramienta gráfica dentro del ecosistema de ROS que permite visualizar una amplia gama de información generada por robots. Esta herramienta es esencial para depurar, entender y visualizar los datos que se generan en tiempo real durante la simulación o la operación de un robot en un entorno físico.

<img src="imagenes/rviz.png" alt = "RViz" height="300" display= "block"/>


RViz soporta muchos tipos de visualizaciones, en este caso veremos:

- **Modelos 3D del robot (URDF)**: Puedes cargar y visualizar la descripción del robot usando su archivo URDF (Unified Robot Description Format). Esto te permite ver un modelo 3D del robot y observar cómo se mueve cada parte del robot (es decir, los eslabones y las juntas). También puedes ver cómo se actualiza en tiempo real cuando el robot cambia de posición.

- **Transformaciones (TF)**: ROS utiliza un sistema de coordenadas llamado TF (transformación de marcos) para definir la relación espacial entre diferentes partes del robot y el entorno. En RViz, puedes visualizar estos sistemas de referencia y las transformaciones entre ellos, lo que es útil para entender cómo se mueven y orientan los diferentes componentes del robot.
Esto incluye visualizar el origen de coordenadas del robot y sus partes móviles.

## Cuestionario previo

### Responder de forma breve las siguientes preguntas:

- ¿Qué es un ejecutor (executor) en ROS2?
> RESPUESTA- Un executor en ROS2 es el componente que se encarga de gestionar y ejecutar los callbacks de los nodos. Recibe los eventos (mensajes en tópicos, llamadas a servicios, timers, etc.) y decide cuándo llamar a las funciones asociadas de cada nodo.
- ¿Qué es la ejecución por hilos en python?
> RESPUESTA- La ejecución por hilos (threading) en Python consiste en crear varios hilos de ejecución dentro de un mismo programa. Cada hilo puede correr una parte del código de forma concurrente y todos comparten la misma memoria del proceso.
- ¿Qué ventajas tiene ejecutar un programa en un hilo secundario?
> RESPUESTA- Ejecutar código en un hilo secundario permite que el hilo principal no se bloquee mientras se realiza una tarea larga o bloqueante. De esta forma, el programa puede seguir respondiendo y es posible manejar varias tareas de manera concurrente.
- ¿Qué pasaría si se utiliza un comando que se queda en periodo de espera (como rclpy.spin()) dentro de un entorno de celdas como jupyter?
> RESPUESTA- La función rclpy.spin() es bloqueante. Si se ejecuta directamente en una celda de Jupyter, el kernel queda ocupado esperando mensajes y no permite ejecutar otras celdas. El notebook parecerá congelado hasta que se detenga el spin o se reinicie el kernel, por lo que normalmente se usa un hilo secundario para evitar este problema.


## Desarrollo

### 1. Tópicos en ROS2

### Importación de librerías y dependencias

En este bloque importamos `rclpy` (la librería cliente de ROS2 para Python) y `std_msgs` para poder enviar mensajes de texto. También importamos `threading` y `MultiThreadedExecutor` para permitir que los nodos se ejecuten en segundo plano sin bloquear el *kernel* de Jupyter.

In [1]:
import rclpy
import time
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor

### Definición del Nodo Publicador

La clase `Publisher` hereda de `Node`. Su función es crear un tópico llamado `mi_topico` y configurar un temporizador (*timer*) que se dispara cada 1.0 segundos. Cada vez que el temporizador se activa, la función `timer_callback` envía un mensaje de texto con un contador incremental.

In [2]:
class Publisher(Node):
 
  def __init__(self):
   
    super().__init__('publicador_simple')
    
    self.publisher_ = self.create_publisher(String, 'mi_topico', 10)
    
    
    timer_period = 1.0 
    self.timer = self.create_timer(timer_period, self.timer_callback)
    self.i = 0
    self.get_logger().info(' Nodo Publicador ha sido iniciado y está publicando.')

  def timer_callback(self):
   
    msg = String()
    msg.data = 'Hola! Este es el mensaje: %d' % self.i
    self.publisher_.publish(msg)
   
    self.get_logger().info('Publicando: "%s"' % msg.data)
    self.i += 1

### Definición del Nodo Suscriptor

La clase `Subscriber` también hereda de `Node`. Se suscribe al mismo tópico (`mi_topico`) para escuchar los mensajes. Se define una función *callback* llamada `listener_callback` que se ejecuta automáticamente cada vez que llega un mensaje nuevo, imprimiendo su contenido en la consola.

In [3]:
class Subscriber(Node):

  def __init__(self):
    
    super().__init__('suscriptor_simple')
  
    self.subscription = self.create_subscription(
      String,
      'mi_topico',
      self.listener_callback,
      10)
    self.subscription  
    self.get_logger().info(' Nodo Suscriptor ha sido iniciado y está escuchando.')

  def listener_callback(self, msg):
    
    
    self.get_logger().info('He recibido: "%s"' % msg.data)

### Ejecución del Sistema

En este bloque se inicializa el sistema ROS2 y se instancia un `MultiThreadedExecutor`. Se utiliza un **hilo secundario** (`threading.Thread`) para correr el ejecutor, lo cual es crucial en Jupyter para evitar que la celda se bloquee indefinidamente.

Se añaden ambos nodos al ejecutor y se deja correr la comunicación durante 5 segundos para visualizar el intercambio de mensajes antes de destruir los nodos y liberar recursos.

In [4]:

if not rclpy.ok():
  rclpy.init(args=None)
  print("Iniciando ROS2...")


executor = MultiThreadedExecutor()

thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()


try:
  pub_node = Publisher()
  sub_node = Subscriber()

  
  executor.add_node(sub_node)
  executor.add_node(pub_node)

  print("Nodos Publicador y Suscriptor añadidos al ejecutor.")
  print("Viendo la comunicación por 10 segundos...")
  
  
  time.sleep(10.0) 
  print("Tiempo de demostración terminado.")

finally:
 
  print("Destruyendo nodos...")
  pub_node.destroy_node()
  sub_node.destroy_node()
 
  print("Nodos destruidos.")

Iniciando ROS2...
Nodos Publicador y Suscriptor añadidos al ejecutor.
Viendo la comunicación por 10 segundos...


[INFO] [1763495622.620239375] [publicador_simple]:  Nodo Publicador ha sido iniciado y está publicando.
[INFO] [1763495622.627861900] [suscriptor_simple]:  Nodo Suscriptor ha sido iniciado y está escuchando.
[INFO] [1763495623.585317602] [publicador_simple]: Publicando: "Hola! Este es el mensaje: 0"
[INFO] [1763495623.588418747] [suscriptor_simple]: He recibido: "Hola! Este es el mensaje: 0"
[INFO] [1763495624.584251336] [publicador_simple]: Publicando: "Hola! Este es el mensaje: 1"
[INFO] [1763495624.588655633] [suscriptor_simple]: He recibido: "Hola! Este es el mensaje: 1"
[INFO] [1763495625.584194782] [suscriptor_simple]: He recibido: "Hola! Este es el mensaje: 2"
[INFO] [1763495625.587865303] [publicador_simple]: Publicando: "Hola! Este es el mensaje: 2"
[INFO] [1763495626.593485257] [publicador_simple]: Publicando: "Hola! Este es el mensaje: 3"
[INFO] [1763495626.596049417] [suscriptor_simple]: He recibido: "Hola! Este es el mensaje: 3"
[INFO] [1763495627.586109424] [suscriptor_si

Tiempo de demostración terminado.
Destruyendo nodos...
Nodos destruidos.


[INFO] [1763495632.588732584] [suscriptor_simple]: He recibido: "Hola! Este es el mensaje: 9"
[INFO] [1763495632.591559827] [publicador_simple]: Publicando: "Hola! Este es el mensaje: 9"


### 2. Servicios en ROS2

### Definición del Nodo Servidor

En ROS2, los servicios funcionan bajo demanda (petición-respuesta). Aquí creamos la clase `Server` que ofrece un servicio llamado `sumar_dos_enteros`.

Utilizamos el tipo de servicio `AddTwoInts` (de la librería `example_interfaces`). Cuando el servidor recibe una petición, la función `add_two_ints_callback` toma los dos números (`a` y `b`), los suma y devuelve el resultado en la respuesta.

In [8]:
from example_interfaces.srv import AddTwoInts
from rclpy.node import Node

class Server(Node):
    def __init__(self):
        super().__init__('servidor_sumador')
    
        self.srv = self.create_service(AddTwoInts, 'sumar_dos_enteros', self.add_two_ints_callback)
        self.get_logger().info('Servidor de sumas iniciado. Listo para recibir peticiones.')

    def add_two_ints_callback(self, request, response):
       
        response.sum = request.a + request.b
        self.get_logger().info('Petición recibida: %d + %d' % (request.a, request.b))
        return response

### Definición del Nodo Cliente

La clase `Client` crea un cliente para conectarse al servicio `sumar_dos_enteros`.

Incluye una función `send_request` que envía los dos números al servidor. Usamos `call_async` para enviar la petición de forma asíncrona, permitiendo que el programa siga ejecutándose mientras espera la respuesta del servidor.

In [9]:
class Client(Node):
    def __init__(self):
        super().__init__('cliente_sumador')
        
        self.cli = self.create_client(AddTwoInts, 'sumar_dos_enteros')
        
        
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('Servidor no disponible, esperando...')
        
        self.req = AddTwoInts.Request()

    def send_request(self, a, b):
        self.req.a = a
        self.req.b = b
        self.get_logger().info('Enviando petición: %d + %d' % (a, b))
        
        
        self.future = self.cli.call_async(self.req)
        return self.future

### Prueba del Servicio: Suma de 7 + 12

En este bloque principal:
1. Instanciamos el Servidor y el Cliente.
2. Los ejecutamos en un hilo secundario.
3. El cliente envía la petición para sumar **7 + 12**.
4. Esperamos la respuesta e imprimimos el resultado.

In [10]:
import time
import threading
from rclpy.executors import MultiThreadedExecutor


server_node = Server()
client_node = Client()


executor_servicios = MultiThreadedExecutor()
executor_servicios.add_node(server_node)
executor_servicios.add_node(client_node)


thread_servicios = threading.Thread(target=executor_servicios.spin, daemon=True)
thread_servicios.start()

try:
    
    future = client_node.send_request(7, 12)
    
   
    while not future.done():
        time.sleep(0.1)

   
    try:
        response = future.result()
        client_node.get_logger().info(
            ' Respuesta recibida: %d + %d = %d' % 
            (client_node.req.a, client_node.req.b, response.sum))
    except Exception as e:
        client_node.get_logger().error('La llamada al servicio falló %r' % (e,))

finally:
   
    server_node.destroy_node()
    client_node.destroy_node()
    print("Prueba de servicios terminada.")

Prueba de servicios terminada.


[INFO] [1763496119.375000945] [servidor_sumador]: Servidor de sumas iniciado. Listo para recibir peticiones.
[INFO] [1763496119.385245993] [cliente_sumador]: Enviando petición: 7 + 12
[INFO] [1763496119.395463490] [servidor_sumador]: Petición recibida: 7 + 12
[INFO] [1763496119.488843198] [cliente_sumador]:  Respuesta recibida: 7 + 12 = 19


### Ejecución desde Terminal

Adicionalmente, se realizó la prueba de estos mismos nodos (Cliente y Servidor) directamente desde la terminal de Ubuntu, utilizando scripts de Python independientes (`servidor.py` y `cliente.py`).

A continuación se muestra la evidencia de dicha ejecución, donde se observa la petición de suma y la respuesta correcta del servidor:

<img src="imagenes/Parte2.jpg" alt="Evidencia Terminal" width="600"/>

### 2. URDF y RViz

### Descripción del Robot (URDF)

Se diseñó un robot de 3 grados de libertad (tipo RRR) utilizando el formato URDF. El robot consta de:
* **Base:** Un bloque negro fijo.
* **Eslabones:** Se utilizaron colores distintivos (Cyan, Rosa, Amarillo) para diferenciar cada segmento.
* **Juntas (Joints):** Se implementaron juntas de tipo `revolute` (rotacionales) y una `prismatic` (lineal) para la mano, todas con límites de movimiento definidos.

A continuación se presenta el código XML del modelo:

```xml
<?xml version="1.0"?>
<robot name="robot_rrr">

  <link name="base_link">
    <visual>
      <geometry>
        <box size="0.3 0.3 0.15"/>
      </geometry>
      <material name="negro">
        <color rgba="0.1 0.1 0.1 1"/>
      </material>
    </visual>
  </link>

  <joint name="shoulder_joint" type="revolute">
    <parent link="base_link"/>
    <child link="eslabon_vertical_cyan"/>
    <origin xyz="0 0 0.4" rpy="0 0 0"/> 
    <axis xyz="0 0 1"/> 
    <limit effort="10.0" lower="-3.14" upper="3.14" velocity="1.0"/>
  </joint>

  <link name="eslabon_vertical_cyan">
    <visual>
      <geometry>
        <box size="0.04 0.04 0.45"/> 
      </geometry>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <material name="cyan">
        <color rgba="0 1 1 1"/>
      </material>
    </visual>
  </link>

  <joint name="junta_fija_rosa" type="fixed">
    <parent link="eslabon_vertical_cyan"/>
    <child link="eslabon_horizontal_rosa"/>
    <origin xyz="0 0.15 0" rpy="0 0 0"/> 
  </joint>

  <link name="eslabon_horizontal_rosa">
    <visual>
      <geometry>
        <box size="0.04 0.3 0.04"/> 
      </geometry>
      <material name="rosa">
        <color rgba="1 0 1 1"/>
      </material>
    </visual>
  </link>

  <joint name="forearm_joint" type="revolute">
    <parent link="eslabon_horizontal_rosa"/>
    <child link="eslabon_horizontal_amarillo"/>
    <origin xyz="0 0.15 0" rpy="0 0 0"/> 
    <axis xyz="0 0 1"/> 
    <limit effort="10.0" lower="-1.57" upper="1.57" velocity="1.0"/>
  </joint>

  <link name="eslabon_horizontal_amarillo">
    <visual>
      <geometry>
        <box size="0.3 0.04 0.04"/> 
      </geometry>
      <origin xyz="0.15 0 0" rpy="0 0 0"/> 
      <material name="amarillo">
        <color rgba="1 1 0 1"/>
      </material>
    </visual>
  </link>

  <joint name="hand_joint" type="prismatic">
    <parent link="eslabon_horizontal_amarillo"/>
    <child link="eslabon_mano_blanco"/>
    <origin xyz="0.3 0 0" rpy="0 0 0"/> 
    <axis xyz="0 0 1"/> 
    <limit effort="10.0" lower="-0.2" upper="0.2" velocity="1.0"/> 
  </joint>

  <link name="eslabon_mano_blanco">
    <visual>
      <geometry>
        <cylinder radius="0.015" length="0.3"/>
      </geometry>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <material name="blanco">
        <color rgba="1 1 1 1"/>
      </material>
    </visual>
  </link>

</robot>
```

### Visualización en RViz

Se ejecutó el paquete `urdf_tutorial` para visualizar el modelo. A continuación se presentan las evidencias de la visualización en RViz2.

En las imágenes se puede apreciar la estructura del robot RRR (colores cyan, rosa y amarillo) y la interfaz gráfica (`joint_state_publisher_gui`) utilizada para manipular las articulaciones en tiempo real.

<div style="display: flex; justify-content: space-around;">
    <img src="imagenes/Robot1.jpg" alt="Vista Robot 1" width="45%"/>
    <img src="imagenes/robot2.jpg" alt="Vista Robot 2" width="45%"/>
</div>


## Análisis de resultados

¿Cuáles son las ventajas o desventajas de usar el protocolo de comunicación publicador/suscriptor ó cliente/servicio?
> RESPUESTA- En la práctica se observaron claramente las diferencias entre ambos esquemas de comunicación. Al implementar los nodos publicador y suscriptor, el protocolo publicador/suscriptor permitió enviar mensajes de forma continua sin necesidad de que los nodos se conocieran entre sí: basta con que compartan el mismo tópico. Esto es ventajoso para flujos de datos constantes, como el envío periódico de información desde un sensor.
Sin embargo, este esquema no proporciona una confirmación directa de que algún nodo haya recibido el mensaje y no es el más adecuado cuando se necesita una respuesta inmediata y única a una petición concreta.
En cambio, con el uso del cliente y el servidor de servicios se tuvo una comunicación tipo petición–respuesta: el cliente envía una solicitud específica y el servidor responde con un resultado. Esto es muy útil para operaciones puntuales (por ejemplo, un cálculo o una orden de movimiento), ya que el cliente sabe exactamente cuándo se atendió la petición. La desventaja es que el cliente depende de que el servidor esté disponible y no es eficiente para datos a alta frecuencia, ya que cada llamada tiene un costo adicional.


¿Cuál es la convención de ángulos que utilizan los archivos URDF para los ángulos de las juntas respecto al eslabón padre? (Intrínsecos/extrínsecos y el orden)
> RESPUESTA- En los archivos URDF la orientación se define mediante tres ángulos RPY (roll, pitch, yaw). Estos ángulos se aplican en el orden roll, luego pitch y finalmente yaw. El roll corresponde a una rotación alrededor del eje X, el pitch a una rotación alrededor del eje Y y el yaw a una rotación alrededor del eje Z.
Se considera que estas son rotaciones extrínsecas con respecto al marco del eslabón padre, en el orden X–Y–Z. Es decir, primero se rota alrededor de X, después alrededor de Y y al final alrededor de Z tomando siempre como referencia los ejes del eslabón padre, lo cual garantiza una convención consistente para todas las juntas del robot.


¿Qué utilidad tiene describir un robot en un archivo URDF?
> RESPUESTA- Describir el robot en un archivo URDF permitió construir un modelo estructurado del manipulador utilizado en la práctica, incluyendo la base, los eslabones vertical y horizontales, la mano y las tres juntas rotacionales. Gracias a esta descripción fue posible visualizar el robot en RViz, manipular sus juntas con el "joint_state_publisher_gui" y verificar que los movimientos correspondieran con la geometría especificada.
En general, el URDF sirve como una representación estándar de la estructura física del robot (eslabones, juntas, geometría, masas y materiales) que puede ser reutilizada por diferentes herramientas como RViz, Gazebo o MoveIt. Esto facilita la simulación, la depuración y el intercambio del modelo entre distintos proyectos.





## Conclusiones

A lo largo de la práctica se logró utilizar las funciones básicas de ROS2 en un entorno interactivo, integrando nodos que se comunican mediante tópicos y servicios. El uso de un executor multihilo permitió ejecutar ROS2 en segundo plano dentro de Jupyter sin bloquear el notebook, lo que hizo posible observar en tiempo real el envío y la recepción de mensajes entre publicadores, suscriptores, clientes y servidores.


Posteriormente, la construcción del modelo URDF del robot manipulador y su despliegue en RViz permitieron visualizar la relación entre eslabones y juntas, así como comprobar el efecto de los límites de cada articulación. Al mover los deslizadores del "joint_state_publisher_gui" se confirmó que las tres juntas rotacionales del robot generan los movimientos esperados sobre la base negra y los eslabones de colores.


En conjunto, la práctica fortaleció la comprensión de los mecanismos de comunicación en ROS2 y de la importancia de describir formalmente un robot mediante URDF. Estos conocimientos son fundamentales para el desarrollo de aplicaciones robóticas más complejas que integren percepción, control y planificación de movimiento.



## Bibliografía 

[1] Open Robotics, "ROS 2 Documentation: Humble Hawksbill," *ROS 2 Documentation*, 2024. [En línea]. Disponible: https://docs.ros.org/en/humble/index.html. [Accedido: 14-nov-2025].

[2] Python Software Foundation, "threading — Thread-based parallelism," *Python 3.12 Documentation*, 2024. [En línea]. Disponible: https://docs.python.org/3/library/threading.html. [Accedido: 14-nov-2025].

[3] Open Robotics, "urdf - XML specifications," *ROS Wiki*, 2018. [En línea]. Disponible: http://wiki.ros.org/urdf/XML. [Accedido: 14-nov-2025].